In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

## Data Loading

In [2]:
import helper
data = helper.load_RWHAR()

from dataset import TimeSeriesDataset
dtset = TimeSeriesDataset(data)

weight = helper.dist(dtset, 8)

train, val = helper.split(dtset, val_pc = 0.3)
train_weight = helper.dist(train, 8)
val_weight = helper.dist(val, 8)
print(train_weight, val_weight)
train_iter = torch.utils.data.DataLoader(train, batch_size = 128, shuffle = True, num_workers = 10, pin_memory = True)
val_iter = torch.utils.data.DataLoader(val, batch_size = 128, num_workers = 10, pin_memory = True)

File exists. Loading
Selecting location :  forearm
Windowing
Done!
[[0.10036932]
 [0.09493808]
 [0.02339054]
 [0.15450069]
 [0.16648563]
 [0.15363169]
 [0.15113332]
 [0.15555073]] [[0.09327475]
 [0.09572491]
 [0.02129098]
 [0.15275431]
 [0.16382224]
 [0.15866847]
 [0.1528388 ]
 [0.16162555]]


In [13]:
from DeepConvLSTM_model import DeepConvNet
from Validation_model import Net
net = DeepConvNet(in_channels = 6,input_size = 50, hidden_size = 512, output_size = 8, conv_filter = (3,5), conv_padding = (1,2))
model = Net(model = net, num_classes = 8, classes_weight = torch.tensor(train_weight, dtype = torch.float), lr = 0.01)

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 5, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'Validation_LSTM_logs/', name = "RWHAR"),
                     stochastic_weight_avg=True
                     )
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DeepConvNet      | 6.2 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.934    Total estimated model params size (MB)


1

In [14]:
from TransformerClassifier import TransformerClassifier
from Validation_model import Net
net = TransformerClassifier(in_channels = 6, output_size = 8, d_model = 50, nhead = 5, dim_feedforward = 8000, num_layers = 5)
model = Net(model = net, num_classes = 8, classes_weight = torch.tensor(train_weight, dtype = torch.float), lr = 0.01)

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 7, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'Validation_transformer_logs/', name = "RWHAR"),
                     stochastic_weight_avg=True
                     )
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name      | Type                  | Params
----------------------------------------------------
0 | model     | TransformerClassifier | 4.1 M 
1 | criterion | CrossEntropyLoss      | 0     
----------------------------------------------------
4.1 M     Trainable params
0         Non-trainable params
4.1 M     Total params
16.532    Total estimated model params size (MB)


1